<a href="https://colab.research.google.com/github/uu-sml/wasp-assigninmen-af-classification/blob/main/assignment_ecg_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WASP Course: Artificial Intelligence and Machine Learning

Lecturer: Dave Zachariah

Assignment responsible: Jingwei Hu, Tianru Zhang, David Vävinggren

# Student and Group Information

Fill this out for the submission of the assignment (you submit this notebook with your solution)

- **Student names:** <font color='red'>Yangyang Wen</font>


- **Team ID:** <font color='red'>YangyangWen_Group25</font>


Make sure that the team id is the same as the one with which you submit your model predictions (see coding task 7) such that we can check your performance.

---
# Module 3 - Assignment Overview: ECG classification

The [electrocardiogram (ECG)](https://www.mayoclinic.org/tests-procedures/ekg/about/pac-20384983) records the electrical signals in the heart. It is a common  test used to quickly detect heart problems and to monitor the heart's health.
In this assignment you will implement and evaluate a model to classify whether the person has [atrial fibrillation (AF)](https://www.mayoclinic.org/diseases-conditions/atrial-fibrillation/symptoms-causes/syc-20350624.) or not based on measurements from the ECG exam.


**Submission:** You submit the deliverables (see below) at https://canvas.kth.se/courses/54581/assignments

**Due Date:** August 22, 2025.

---
## Basic Tasks
Your task is to implement a classification model, train this model on training data, and evaluate its performance on validation data. We provide skeleton code for the implementation of a simple convolution neural network model.

The steps required to implement this model are presented as numbered tasks below. In total there are seven (7) coding tasks and five (5) explanation tasks.

## Competitive setting

You have to compute the predictions for the test data (you do not have the labels for it) and submit your predictions to be evaluated to a leaderboard. These predictions will be scored and your submission will be ranked according to the F1 score and compared with your colleagues. In the end a winning team will be determined.

### Deliverables
There are two deliverables:
1. You have to submit this Jupyter notebook on the course web-page (Canvas) together with your code and explanations (where asked for it) that describe your implementation and your experimental results. The notebook should run as a standalone in google colab.
2. You have to have at least **three (3)** submissions (for instructions on how to submit, see coding task 7) where you try to improve the model architecture, the training procedure or the problem formulation. In the submission of this notebook you have to provide a short explanation of what changed between each submission and justify why you decided to make these changes.

### Grading
To pass the assignment, you must submit a complete and working implementation of a model and a well-motivated description and evaluation of it. Your model should reach an Area under the ROC curve (AUROC) on the test data of at least 0.97 and an Average Precision (AP) score of 0.95. Note that the leaderboard to is sorted by F1 score and not AUROC, hence you would want to balance all three metrics.

### GPU Acceleration
To be able to use the GPUs provided by colab in order to speed up your computations, you want to check that the `Hardware accelerator` is set to `GPU` under `Runtime > change runtime type`. Note that notebooks run by connecting to virtual machines that have maximum lifetimes that can be as much as 12 hours. Notebooks will also disconnect from VMs when left idle for too long.

In [ ]:
import os

# helper function
def exists(path):
    val = os.path.exists(path)
    if val:
        print(f'{path} already exits. Using cached. Delete it manually to recieve it again!')
    return val

# clone requirements.txt if not yet available
if not exists('requirements.txt'):
    !git clone https://gist.github.com/dgedon/8a7b91714568dc35d0527233e9ceada4.git req
    !mv req/requirements.txt .
    !yes | rm -r req

requirements.txt already exits. Using cached. Delete it manually to recieve it again!


In [ ]:
# Install packages (python>=3.9 is required)
!pip3 install -r requirements.txt

In [ ]:
# Import
import torch
import torch.nn as nn
import numpy as np
from tqdm.notebook import trange, tqdm
import h5py
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

---
## The data set

The dataset is a subset of the [*CODE dataset*](https://scilifelab.figshare.com/articles/dataset/CODE_dataset/15169716): an anotated database of ECGs. The ECG exams were recorded in Brazil by the Telehealth Network of the state Minas Gerais between 2010 and 2016. The dataset and its usage for the development of deep learning methods was described in ["Automatic diagnosis of the 12-lead ECG using a deep neural network"](https://www.nature.com/articles/s41467-020-15432-4).
The full dataset is available for research upon request.


For the training dataset you have labels.
For the test dataset you only have the ECG exams but no labels. Evaluation is done by submitting to the leaderboard.

Download the dataset from the given dropbox link and unzip the folder containing the files. The downloaded files are in WFDB format (see [here](https://www.physionet.org/content/wfdb-python/3.4.1/) for details).

In [ ]:
# 1. Download dataset
if not exists('codesubset.tar.gz'):
    !wget https://www.dropbox.com/s/9zkqa5y5jqakdil/codesubset.tar.gz?dl=0 -O codesubset.tar.gz

In [ ]:
# 1. unzip the downloaded data set folder
if not exists('codesubset'):
    !tar -xf codesubset.tar.gz

Note that the extraced folder 'codesubset' contains
1. subfolders with the ECG exam traces. These have to be further preprocessed which we do in the next steps.
2. a csv file which contain the labels and other features for the training data set.


### Preprocessing

Run the cells below to  Clone the GitHub repository which we use for [data preprocessing](https://github.com/antonior92/ecg-preprocessing).

In [ ]:
# 2. clone the code files for data preprocessing
if not exists('ecg-preprocessing'):
    !git clone https://github.com/paulhausner/ecg-preprocessing.git

Let us plot an ECG sample. We can plot ECGs using the `ecg_plot` library for example by using the following code snippet where `ecg_sample` is an array of size `(number of leads * sequence length)`. Now we can view an ECG before preprocessing.

In [ ]:
import ecg_plot
runfile("ecg-preprocessing/read_ecg.py")

PATH_TO_WFDB = 'codesubset/train/TNMG100046'
ecg_sample, sample_rate, _ = read_ecg(PATH_TO_WFDB)

# ECG plot
plt.figure()
lead = ['I', 'II', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
ecg_plot.plot(ecg_sample, sample_rate=sample_rate, style='bw', row_height=8, lead_index=lead, columns=1, title='Sample ECG before pre-processing')
plt.show()


The preprocessing consist of:
- resampling all ECG traces to the sample sampling period (400 Hz). Option: ``--new_freq 400``
- zero padding if necessary such that all ECG have the same number of samples (4096). Option: ``--new_len 4096``.
- removing trends in the ECG signal. Option: ``--remove_baseline``
- remove possible power line noise. Option: ``--powerline 60``

You can run the script bellow to plot the same ECG after the preprocessing.  The script also use the  `ecg_plot` library (as you did above).  You can try also with different command line options to see how the preprocessing affects the signal that will be used by the model.

In [ ]:
%run ecg-preprocessing/plot_from_ecg.py codesubset/train/TNMG100046 --new_freq 400 --new_len 4096 --remove_baseline --powerline 60


Next we perform the preprocessing in all exams and convert them into one single h5 file (see [here](https://www.h5py.org/#:~:text=The%20h5py%20package%20is%20a,they%20were%20real%20NumPy%20arrays.) for details about the format). The resulting h5 files contains the traces as arrays with the shape `(number of traces * sequence length * number of leads)` where sequence length is 4096 and number of leads is 8.
The files `train.h5` and `test.h5` will be saved inside the folder `codesubset/`.

In [ ]:
# 3. Generate train
if not exists('codesubset/train.h5'):
    !python ecg-preprocessing/generate_h5.py --new_freq 400 --new_len 4096 --remove_baseline --powerline 60 codesubset/train/RECORDS.txt codesubset/train.h5
# 3. Generate test
if not exists('codesubset/test.h5'):
    !python ecg-preprocessing/generate_h5.py --new_freq 400 --new_len 4096 --remove_baseline --powerline 60 codesubset/test/RECORDS.txt codesubset/test.h5

### Coding Task 1: Data Analysis

Before starting to model you have to analyse the dataset. You can be creative in your way of *getting a feeling* for the data. What you have to do is:
- plot an ECG after proprocessing saved in the hdf5 file. For this use the `ecg_plot()` example above and see below for how to access the preprocessed data in h5 format.

Some further ideas to explore are:
- check the balance of the data set,
- evaluate the distribution of age and sex of the patients,
- think about the performance that a best naive classifier would achieve, e.g. by random guessing or always predicting one class.

<br />

**How to access the data?**

You can acces the data in the h5 file in the following way
```
import h5py

PATH_TO_H5_FILE = 'codesubset/train.h5'
f = h5py.File(PATH_TO_H5_FILE, 'r')
data = f['tracings']
```
Then, `data[i]` is an numpy array of the $i$th ECG exam (including all time points and leads).


### Explanation task 1: Data Analysis

Please explain your main findings of the data analysis task in a few bullet points. Explain also what the preprocessing does and why it is necessary.

<br />


**<font color='red'>Your explanation here:</font>**

Main findings from data analysis:

By comparing ECG plots before and after preprocessing, I found that (1) the waveform cycles in the traces become wider and stretched after preprocessing, reflecting the original sampling rates like 250 Hz being unified to 400 Hz; (2) zero-padding was applied at the beginning and end of all 8 leads to ensure the completeness and uniform length of the data.

What preprocessing does:
① Resampling: all ECG traces are resampled to 400 Hz, resulting in 4096 data points for a 10-second segment.
② Zero padding: for signals shorter than 10 seconds, zeros are added at the start and end to reach 4096 points, satisfying the fixed input size required by neural networks.
③ Baseline removal: trends and slow drifts in the ECG signal are removed, resulting in a more stable baseline that helps in extracting true cardiac features.
④ Powerline noise removal: 60 Hz electrical interference is filtered out to reduce environmental noise and improve signal quality.

Why preprocessing is necessary:
① To ensure all traces have consistent sampling rates and lengths for model training.
② To remove noise and baseline drift, improving signal quality and enabling the model to learn meaningful features.
③ To reduce environmental interference, making model predictions more stable and accurate.

In [ ]:
"""
TASK: Insert your code here
"""
import h5py

import matplotlib.pyplot as plt

PATH_TO_H5_FILE = 'codesubset/train.h5'
f = h5py.File(PATH_TO_H5_FILE, 'r')
data = f['tracings']

# Take 0th sample
ecg_sample= data[0] # shape is (4096, 8), but ecg_plot needs (number_of_leads, sequence_length), here is one sample

ecg_sample = ecg_sample.T

plt.figure()
lead = ['I', 'II', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
ecg_plot.plot(ecg_sample, sample_rate=400, style='bw', row_height=8, lead_index=lead, columns=1, title='Sample ECG after preprocessing')
plt.show()

In [ ]:
# 1. Calculate the proportion of each label (data balance)
import pandas as pd

# read label CSB
df = pd.read_csv('codesubset/train.csv')
#print(df)
# Calculate the number of each category
label_counts = df['AF'].value_counts()
print(label_counts)
# Caculate the ratio
label_ratio = label_counts / len(df)
print(label_ratio)

In [ ]:
# 2. Analyze age and gender distribution

import matplotlib.pyplot as plt

# Age distribution histogram
plt.hist(df['age'].dropna(), bins=30, color='skyblue')
plt.title('Age distribution')
plt.xlabel('Age')
plt.ylabel('Count')
plt.show()

# Gender distribution (assuming the sex column is represented by 0 and 1)
sex_counts = df['sex'].value_counts()
sex_counts.index = ['Female', 'Male'] # Modify based on actual labels
sex_counts.plot(kind='bar', color=['pink', 'lightblue'])
plt.title('Sex distribution')
plt.ylabel('Count')
plt.show()

In [ ]:
# 3. Simple "naive baseline classifier"

from sklearn.metrics import accuracy_score, f1_score

# true labels
y_true = df['AF'].values

# baseline prediction: predict all as 0
y_pred = [0]*len(y_true) # [1] is tried as well

acc = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f'Baseline Accuracy: {acc:.4f}')
print(f'Baseline F1 Score: {f1:.4f}')
# This shows the model completely failed to predict positive examples such as "atrial fibrillation" (AF=1) (both recall and precision are 0), resulting in an F1 score of zero.

In [ ]:
# 4. Comparison analysis before and after preprocessing

import h5py
import wfdb
import numpy as np
import matplotlib.pyplot as plt

# === 1. Read ECG samples before preprocessing (WFDB format) ===
PATH_TO_WFDB = 'codesubset/train/TNMG100046'
record = wfdb.rdrecord(PATH_TO_WFDB)
ecg_before = record.p_signal.T  # (leads, length)
fs_before = record.fs

# === 2. Read ECG samples before preprocessing（h5 files）===
PATH_TO_H5_FILE = 'codesubset/train.h5'
f = h5py.File(PATH_TO_H5_FILE, 'r')
ecg_after = f['tracings'][0].T  # (leads, length)

fs_after = 400  # Specify preprocessing parameters --new_freq 400

# === 3. Select a lead（e.g., lead 0）for comparison ===
lead = 0
signal_before = ecg_before[lead]
signal_after = ecg_after[lead]

# === 4. Waveform comparison ===
plt.figure(figsize=(12, 4))
plt.plot(signal_before, label='Before preprocessing')
plt.plot(signal_after, label='After preprocessing')
plt.legend()
plt.title(f"Lead {lead} waveform comparison")
plt.show()

# === 5. Difference waveform ===
#plt.figure(figsize=(12, 4))
#plt.plot(signal_before[:len(signal_after)] - signal_after, color='red')
#plt.title(f"Lead {lead} difference (before - after)")
#plt.show()

# === 6. Spectrum comparison ===
def plot_fft(signal, fs, label):
    N = len(signal)
    freqs = np.fft.rfftfreq(N, 1/fs)
    fft_vals = np.abs(np.fft.rfft(signal))
    plt.plot(freqs, fft_vals, label=label)

plt.figure(figsize=(12, 4))
plot_fft(signal_before, fs_before, "Before")
plot_fft(signal_after, fs_after, "After")
plt.xlim(0, 100)  # Focus on the low-frequency band to easily identify the power frequency (60Hz) position
plt.legend()
plt.title(f"Lead {lead} frequency spectrum comparison")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Amplitude")
plt.show()

In [ ]:
from scipy.signal import resample

signal_before_resampled = resample(signal_before, 4096)

plt.figure(figsize=(12, 4))
plt.plot(signal_before_resampled - signal_after, color='red')
plt.title(f"Lead {lead} difference (before - after)")
plt.show()


---
## Model

The model class consists of two methods:
- `__init__(self, args)`: This methods initializes the class, e.g. by using `mymodel=ModelBaseline(args)`.
- `forward(self,input_data)`: This method is called when we run `model_output=mymodel(input_data)`.

The dimension of the input data is  `(batch size * sequence length * number of leads)`. Where **batch size** is a hyperparameter, **sequence length** is the number of ECG time samples (=4096) and **number of leads** (=8).

The `ModelBaseline` (provided below) is a 2 layer model with one convolutional layers and one linear layer. Some explanations:
- The conv layer downsamples the input traces from 4096 samples to 128 samples and increases the number of channels from 8 (=number of leads) to 32. Here we use a kernel size of 3.
- The linear layer uses the flattened output from the conv and outputs one prediction. Since we have a binary problem, a single prediction is sufficient.


In [ ]:
class ModelBaseline(nn.Module):
    def __init__(self,):
        super(ModelBaseline, self).__init__()
        self.kernel_size = 3

        # conv layer
        downsample = self._downsample(4096, 128)
        self.conv1 = nn.Conv1d(in_channels=8,
                               out_channels=32,
                               kernel_size=self.kernel_size,
                               stride=downsample,
                               padding=self._padding(downsample),
                               bias=False)

        # linear layer
        self.lin = nn.Linear(in_features=32*128,
                             out_features=1)

        # ReLU
        self.relu = nn.ReLU()

    def _padding(self, downsample):
        return max(0, int(np.floor((self.kernel_size - downsample + 1) / 2)))

    def _downsample(self, seq_len_in, seq_len_out):
        return int(seq_len_in // seq_len_out)


    def forward(self, x):
        x= x.transpose(2,1)

        x = self.relu(self.conv1(x))
        x_flat= x.view(x.size(0), -1)
        x = self.lin(x_flat)

        return x

### Coding Task 2: Define your model

In the cell below you have to define your model. You can be inspired by the baseline model above but you can also define any other kind of neural network architecture.

In [ ]:
class Model(nn.Module):
    def __init__(self, num_leads=8):
        super(Model, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(in_channels=num_leads, out_channels=32, kernel_size=5, padding=2),
            nn.BatchNorm1d(32),
            nn.ReLU()
        )

        self.branch1 = nn.Conv1d(32, 64, kernel_size=3, padding=1)
        self.branch2 = nn.Conv1d(32, 64, kernel_size=15, padding=7)
        self.branch3 = nn.Conv1d(32, 64, kernel_size=25, padding=12)

        self.bn = nn.BatchNorm1d(64*3)
        self.relu = nn.ReLU()
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(64*3, 1)

    def forward(self, x):
        x = x.transpose(1,2)
        x = self.conv1(x)
        x = torch.cat([self.branch1(x), self.branch2(x), self.branch3(x)], dim=1)
        x = self.bn(x)
        x = self.relu(x)
        x = self.pool(x).squeeze(-1)
        return self.fc(x)

In [ ]:
class LightCNN(nn.Module):
    def __init__(self, num_leads=8):
        super(LightCNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(num_leads, 16, kernel_size=5, padding=2),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.MaxPool1d(2)  # Downsampling reducce the amount of computation required
        )

        self.branch1 = nn.Conv1d(16, 32, kernel_size=3, padding=1)
        self.branch2 = nn.Conv1d(16, 32, kernel_size=7, padding=3)  # reduce convolution kernels
        self.branch3 = nn.Conv1d(16, 32, kernel_size=11, padding=5)

        self.bn = nn.BatchNorm1d(32*3)
        self.relu = nn.ReLU()
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(32*3, 1)

    def forward(self, x):
        x = x.transpose(1, 2)
        x = self.conv1(x)
        x = torch.cat([self.branch1(x), self.branch2(x), self.branch3(x)], dim=1)
        x = self.bn(x)
        x = self.relu(x)
        x = self.pool(x).squeeze(-1)
        return self.fc(x)

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1, stride=stride)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm1d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv1d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm1d(out_channels)
            )

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = self.relu(out)
        return out

class ResNet1D(nn.Module):
    def __init__(self, num_leads=8):
        super(ResNet1D, self).__init__()
        self.layer1 = ResidualBlock(num_leads, 16)
        self.layer2 = ResidualBlock(16, 32, stride=2)
        self.layer3 = ResidualBlock(32, 64, stride=2)
        self.pool = nn.AdaptiveAvgPool1d(1)
        #self.dropout = nn.Dropout(p=0.3) # My Design
        self.fc = nn.Linear(64, 1)

    def forward(self, x):
        x = x.transpose(1, 2)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.pool(x).squeeze(-1)
        #x = self.dropout(x) # My Design
        return self.fc(x)

In [ ]:
import torch
import torch.nn as nn

# SE attention module(Squeeze-and-Excitation)
class SEBlock(nn.Module):
    def __init__(self, channels, reduction=16):
        super(SEBlock, self).__init__()
        self.fc1 = nn.Linear(channels, channels // reduction, bias=False)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(channels // reduction, channels, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # x: [batch, channels, time]
        y = x.mean(-1)  # Global Avg Pool -> [batch, channels]
        y = self.relu(self.fc1(y))
        y = self.sigmoid(self.fc2(y))
        y = y.unsqueeze(-1)  # [batch, channels, 1]
        return x * y  # Channel weighting

# Upgraded residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, kernel_size=5, dropout=0.2):
        super(ResidualBlock, self).__init__()
        padding = kernel_size // 2
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=kernel_size, padding=padding, stride=stride)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=kernel_size, padding=padding)
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.dropout = nn.Dropout(dropout)
        self.se = SEBlock(out_channels)  # attention module

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv1d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm1d(out_channels)
            )

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.dropout(out)
        out = self.bn2(self.conv2(out))
        out = self.se(out)  # attention
        out += self.shortcut(x)
        out = self.relu(out)
        return out

# Upgraded ResNet1D
class ResNet1D_Improved(nn.Module):
    def __init__(self, num_leads=8, num_classes=1):
        super(ResNet1D_Improved, self).__init__()
        self.layer1 = ResidualBlock(num_leads, 32, kernel_size=7, dropout=0.1)  # bigger convolution kernels
        self.layer2 = ResidualBlock(32, 64, stride=2, kernel_size=5, dropout=0.2)
        self.layer3 = ResidualBlock(64, 128, stride=2, kernel_size=5, dropout=0.3)
        self.layer4 = ResidualBlock(128, 128, stride=2, kernel_size=3, dropout=0.3)  # Add a new layer
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(128, num_classes)

    def forward(self, x):
        x = x.transpose(1, 2)  # [B, Leads, Time]
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.pool(x).squeeze(-1)
        return self.fc(x)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define BasicBlock（2-layer convolution + residual connection）
class BasicBlock1D(nn.Module):
    expansion = 1  # The bottleneck layer is generally 4, but here it is BasicBlock, so it is 1.

    def __init__(self, in_channels, out_channels, stride=1, downsample=None, dropout=0.0):
        super(BasicBlock1D, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout(dropout)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.downsample = downsample  # Used to change the dimension matching residuals
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.dropout(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


# The main structure of ResNet32
class ResNet1D32(nn.Module):
    def __init__(self, block=BasicBlock1D, layers=[5,5,5], num_classes=1, num_leads=8, dropout=0.2):
        super(ResNet1D32, self).__init__()
        self.in_channels = 16

        # Input channel transformation layer
        self.conv1 = nn.Conv1d(num_leads, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm1d(16)
        self.relu = nn.ReLU(inplace=True)

        # Three stages, each stage containing several BasicBlocks
        self.layer1 = self._make_layer(block, 16, layers[0], stride=1, dropout=dropout)
        self.layer2 = self._make_layer(block, 32, layers[1], stride=2, dropout=dropout)  # downsample
        self.layer3 = self._make_layer(block, 64, layers[2], stride=2, dropout=dropout)  # downsample

        # Global average pooling
        self.avgpool = nn.AdaptiveAvgPool1d(1)
        # classification layer
        self.fc = nn.Linear(64 * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride=1, dropout=0.0):
        downsample = None
        if stride != 1 or self.in_channels != out_channels * block.expansion:
            downsample = nn.Sequential(
                nn.Conv1d(self.in_channels, out_channels * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm1d(out_channels * block.expansion),
            )

        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample, dropout=dropout))
        self.in_channels = out_channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels, dropout=dropout))

        return nn.Sequential(*layers)

    def forward(self, x):
        # x: [batch, leads, time]
        x = x.transpose(1, 2)  # change to [batch, channels, length]

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.avgpool(x).squeeze(-1)  # [batch, channels]
        x = self.fc(x)

        return x

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, num_leads=8, hidden_size=64, num_layers=2):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(num_leads, hidden_size, num_layers=num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, 1)

    def forward(self, x):
        # x: (batch, seq_len, num_leads)
        out, _ = self.lstm(x)
        out = out[:, -1, :]  # last moment
        return self.fc(out)

In [ ]:
class TransformerModel(nn.Module):
    def __init__(self, num_leads=8, d_model=64, nhead=4, num_layers=2):
        super(TransformerModel, self).__init__()
        self.input_proj = nn.Linear(num_leads, d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(d_model, 1)

    def forward(self, x):
        x = self.input_proj(x)
        x = self.transformer(x)
        x = x.mean(dim=1)
        return self.fc(x)

In [ ]:
class RNNModel(nn.Module):
    def __init__(self, num_leads=8, hidden_size=64, num_layers=1):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(num_leads, hidden_size, num_layers=num_layers, batch_first=True, nonlinearity='tanh')
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = out[:, -1, :]
        return self.fc(out)

### Explanation Task 2: Final Model
Please explain and motivate in short sentences or bullet points the choice of your final model.

<br />


**<font color='red'>Your explanation here:</font>**

I selected a CNN (Convolutional Neural Network) as the final model. According to the question requirements, I focused on improving the F1 score while also considering AUROC and Average Precision to balance precision and recall, ensuring the model performs well in distinguishing positive and negative samples. The reasons for choosing this model include:

1️⃣ Multi-scale convolutions (kernel sizes = 3, 15, 25) designed in parallel, enabling the simultaneous capture of both local and long-term dependency features of ECG signals, which is suitable for the multi-frequency characteristics of electrocardiograms.
2️⃣ Incorporation of BatchNorm and ReLU to enhance training stability and nonlinear representation capabilities.
3️⃣ Use of global pooling and fully connected layers with moderate parameter counts to improve model generalization.
4️⃣ Direct output of sigmoid probabilities for convenient threshold adjustment in downstream tasks.
5️⃣ I also tested LSTM, RNN, and Transformer models on this dataset. I found that LSTM and RNN achieved lower accuracy than the CNN, making them less suitable for this scenario. The Transformer model was much slower to train and ran out of memory on this dataset.

This approach is scalable to integrated learning across different architectures, facilitating future optimizations.


---
## Train function

The function `train(...)` is called to in every epoch to train the model. The function loads the training data, makes predictions, compares predictions with true labels in the loss function and adapting the model parameters using stochastic gradient descent.

In the code cell below there is the basic structure to load data from the data loader and to log your loss. The arguments of the function are explained by the use in the `main(...)` function below.

If you are unfamiliar with PyTorch training loops, then this official [tutorial](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html) might help (especially section "4. Train your Network").

### Coding Task 3: Fill training loop

Fill the code cell below such that the model is training when `train(...)` is called.

In [ ]:
def train_loop(epoch, dataloader, model, optimizer, loss_function, device):
    # model to training mode (important to correctly handle dropout or batchnorm layers)
    model.train()
    # allocation
    total_loss = 0  # accumulated loss
    n_entries = 0   # accumulated number of data points
    # progress bar def
    train_pbar = tqdm(dataloader, desc="Training Epoch {epoch:2d}".format(epoch=epoch), leave=True)
    # training loop
    for traces, diagnoses in train_pbar:
        # data to device (CPU or GPU if available)
        traces, diagnoses = traces.to(device), diagnoses.to(device)

        """
        TASK: Insert your code here. This task can be done in 5 lines of code.
        """
        optimizer.zero_grad() # Clear gradient
        outputs = model(traces) # Forward propagation Calculate output
        loss = loss_function(outputs, diagnoses) # Calculate loss
        loss.backward() # Backward propagation Calculate gradient
        optimizer.step() # Update model parameters

        # Update accumulated values
        total_loss += loss.detach().cpu().numpy()
        n_entries += len(traces)

        # Update progress bar
        train_pbar.set_postfix({'loss': total_loss / n_entries})
    train_pbar.close()
    return total_loss / n_entries

---
## Eval function

The `eval(...)` function is similar to the `train(...)` function but is used to evaluate the model on validation data without adapting the model parameters. You can prohibit computing gradients by using a `with torch.no_grad():` statement.

Currenlty only the loss is logged here. Additionally you have to collect all your predictions and the true values in order to compute more metrics such as AUROC.

### Coding Task 4: Fill evaluation loop
Fill the code cell below such we obtain model predictions to evaluate the validation loss and collect the predictoin in order to compute other validation metrics in the `main(...)` function.

In [ ]:
def eval_loop(epoch, dataloader, model, loss_function, device):
    # model to evaluation mode (important to correctly handle dropout or batchnorm layers)
    model.eval()
    # allocation
    total_loss = 0  # accumulated loss
    n_entries = 0   # accumulated number of data points
    valid_probs = []  # accumulated predicted probabilities
    valid_true = [] # accumulated true labels

    # progress bar def
    eval_pbar = tqdm(dataloader, desc="Evaluation Epoch {epoch:2d}".format(epoch=epoch), leave=True)
    # evaluation loop
    for traces_cpu, diagnoses_cpu in eval_pbar:
        # data to device (CPU or GPU if available)
        traces, diagnoses = traces_cpu.to(device), diagnoses_cpu.to(device)

        """
        TASK: Insert your code here. This task can be done in 6 lines of code.
        """
        with torch.no_grad():
          outputs = model(traces)
          loss = loss_function(outputs, diagnoses)

          probs = torch.sigmoid(outputs)  # change to probs
          valid_probs.append(probs.cpu().numpy())
          valid_true.append(diagnoses.cpu().numpy())


        # Update accumulated values
        total_loss += loss.detach().cpu().numpy()
        n_entries += len(traces)

        # Update progress bar
        eval_pbar.set_postfix({'loss': total_loss / n_entries})
    eval_pbar.close()

    valid_probs = np.concatenate(valid_probs, axis=0).squeeze() # The shape of the result is (N, 1)
    valid_true = np.concatenate(valid_true, axis=0).squeeze() # The shape of the result is (N,)

    return total_loss / n_entries, valid_probs, valid_true #np.vstack(valid_probs), np.vstack(valid_true)

---
## Run Training

In the code cell below there are some initial (non-optimal!) training hyperparameters. Further, we combine everything from above into training code. That means that we build the dataloaders, define the model/loss/optimizer and then train/validate the model over multiple epochs. Here, we save the model with the lowest validation loss as the best model.

### Coding Task 5: Combine everything to train/validate the model

The following tasks are necessary in the code below
- split the data into training and validation data
- define the loss function
- decide and implement validation metric(s) to evaluate and compare the model on

Optional task:
- include learning rate scheduler
- take specific care about possible data inbalance

### Coding Task 6: Run your model and adapt hyperparameters

After you combined everything in task 5, now you run the code to evaluate the model. Based on the resulting validation metrics you tune
- the training hyperparameters
- the model architecture
- the model hyperparameters.

### Explanation Task 3: Hyperparameter
Please explain and motivate in short sentences or bullet points the final choice of hyperparamer and how you developed them.

<br />


**<font color='red'>Your explanation here:</font>**
Hyperparameter selection and motivation for choosing the values:
- Learning rate:
  - Determines the magnitude of parameter updates.
  - To large -> raining loss fluctuates, difficult to converge; may cause gradient explosion in deep networks, resulting in NaN or overflow.
  - Too small -> convergence is very slow, may get stuck in high loss regions, low training efficiency, and cannot reach low loss or high accuracy.

- Weight decay:
  - Controls the magnitude of model weights.
  - Larger weight decay reduces weight size, preventing overfitting, but too large may cause underfitting (over-regularization).
  - Chosen to balance between preventing overfitting and maintaining model capacity.

- Num_epochs:
  - Determines how long the model is trained. The choice is based on observing training and validation metrics:
  - (1)Training loss and validation metrics (F1, AUROC) should stabilize
  - (2)Early stopping can be used to avoid overfitting and automatically select optimal epoch.
  - (3)Too few epochs → underfitting; too many epochs → overfitting and wasted computation.
- Batch size:
  - Balances gradient estimation stability and memory usage.
  - Smaller batch size → noisier gradients, more fluctuation during training.
  - Larger batch size → smoother gradients and stable training, but requires more memory to store inputs, gradients, and intermediate activations; may cause out-of-memory errors if too large.


In [ ]:
# set seed
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

# choose variables
"""
TASK: Adapt the following hyperparameters if necessary
"""
learning_rate = 5e-4 #1e-3 #1e-3 #5e-4 #2e-4 #from 5e-4 to 2e-4
weight_decay = 1e-1 #1e-1 #1e-2 #1e-1 # 1e-1 too big, change to 1e-4
num_epochs = 15 #40 #25
batch_size = 32 #64 #32



In [ ]:
from torch.utils.data import TensorDataset, random_split, DataLoader

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tqdm.write("Use device: {device:}\n".format(device=device))

# =============== Build data loaders ======================================#
tqdm.write("Building data loaders...")

path_to_h5_train, path_to_csv_train, path_to_records = 'codesubset/train.h5', 'codesubset/train.csv', 'codesubset/train/RECORDS.txt'
# load traces
traces = torch.tensor(h5py.File(path_to_h5_train, 'r')['tracings'][()], dtype=torch.float32)
# load labels
ids_traces = [int(x.split('TNMG')[1]) for x in list(pd.read_csv(path_to_records, header=None)[0])] # Get order of ids in traces
df = pd.read_csv(path_to_csv_train)
df.set_index('id_exam', inplace=True)
df = df.reindex(ids_traces) # make sure the order is the same
labels = torch.tensor(np.array(df['AF']), dtype=torch.float32).reshape(-1,1)
# load dataset
dataset = TensorDataset(traces, labels)
len_dataset = len(dataset)
n_classes = len(torch.unique(labels))
# split data
"""
TASK: Split the dataset in train and validation; Insert your code here.
This can be done in <=4 line of code
"""
train_size = int(0.8 * len_dataset)
valid_size = len_dataset - train_size
dataset_train, dataset_valid = random_split(dataset, [train_size, valid_size])
# build data loaders
train_dataloader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(dataset_valid, batch_size=batch_size, shuffle=False)
tqdm.write("Done!\n")

In [ ]:
"""
My Design: Data Argumentation

"""
import torch
from torch.utils.data import Dataset
import random

class ECGDatasetWithAug(Dataset):
    def __init__(self, traces, labels, augment=False):
        """
        traces: Tensor, shape [N, Time, Channels] or [N, Channels, Time] ajust according to your data structure
        labels: Tensor, shape [N, 1]
        augment: bool, whether to perform data argumentation 
        """
        self.traces = traces
        self.labels = labels
        self.augment = augment

    def __len__(self):
        return len(self.traces)

    def __getitem__(self, idx):
        x = self.traces[idx].clone()  # clone prevent modifying original data
        y = self.labels[idx]

        if self.augment:
            x = self.augment_signal(x)

        return x, y

    def augment_signal(self, x):
        # Assume that the shape of x is [Channels, Time] or [Time, Channels]，it is written as [Channels, Time] here. Please note that this corresponds to your actual data.
        # 1. Random Gaussian noice
        noise_level = 0.01
        noise = torch.randn_like(x) * noise_level
        x = x + noise

        # 2. Random time shift (cyclic signal shift)
        max_shift = 10  # Maximum time step shift
        shift = random.randint(-max_shift, max_shift)
        if shift != 0:
            x = torch.roll(x, shifts=shift, dims=-1)

        return x



from torch.utils.data import TensorDataset, random_split, DataLoader

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tqdm.write("Use device: {device:}\n".format(device=device))

# ================================ Build data loaders ======================================#
tqdm.write("Building data loaders...")

path_to_h5_train, path_to_csv_train, path_to_records = 'codesubset/train.h5', 'codesubset/train.csv', 'codesubset/train/RECORDS.txt'
# load traces
traces = torch.tensor(h5py.File(path_to_h5_train, 'r')['tracings'][()], dtype=torch.float32)
print(f"trace type:{traces.shape}")
# load labels
ids_traces = [int(x.split('TNMG')[1]) for x in list(pd.read_csv(path_to_records, header=None)[0])] # Get order of ids in traces
df = pd.read_csv(path_to_csv_train)
df.set_index('id_exam', inplace=True)
df = df.reindex(ids_traces) # make sure the order is the same
labels = torch.tensor(np.array(df['AF']), dtype=torch.float32).reshape(-1,1)



# load dataset
# What is the original shape of the traces? Assume it is [N, Time, Channels].
# The original model first transposed it, so the data should be [N, Time, Channels].
# Here, change it to [N, Channels, Time] for convenient convolution, or adjust it according to your data structure.

#traces = traces.permute(0, 2, 1)  # [N, Channels, Time]
#print(f"trace type after permute:{traces.shape}")
# Custom dataset
full_dataset = ECGDatasetWithAug(traces, labels, augment=False)
train_size = int(0.8 * len(full_dataset))
valid_size = len(full_dataset) - train_size
dataset_train, dataset_valid = random_split(full_dataset, [train_size, valid_size])
# Turn on the augmentation switch for the training set.
dataset_train.dataset.augment = True
dataset_valid.dataset.augment = False

train_dataloader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(dataset_valid, batch_size=batch_size, shuffle=False)
tqdm.write("Done!\n")

In [ ]:
import numpy as np
import torch

"""
My design: Search for the threshold that maximizes the F1 score based on the unique values among all predicted probabilities (y_prob) as the threshold.
"""
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

def find_best_threshold_by_f1_exact(y_true, y_prob):
    # Obtain the unique values from all predicted probabilities and sort them (from smallest to largest)
    thresholds = np.sort(np.unique(y_prob))

    best_t = 0.5
    best_f1 = -1
    best_prec = 0
    best_rec = 0

    for t in thresholds:
        y_bin = (y_prob >=t).astype(int)
        prec, rec, f1, _ = precision_recall_fscore_support(y_true, y_bin, average='binary', zero_division=0)
        if f1 > best_f1:
            best_f1 = f1
            best_t = t
            best_prec = prec
            best_rec = rec
    return best_t, best_f1, best_prec, best_rec

class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt'):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
            verbose (bool): If True, prints a message for each validation loss improvement.
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
            path (str): Path for the checkpoint to be saved to.
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} → {val_loss:.6f}).  Saving model...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss


In [ ]:
# =============== Define model ============================================#
tqdm.write("Define model...")
"""
TASK: Replace the baseline model with your model; Insert your code here
"""
model = ResNet1D_Improved() #ResNet1D32() #ModelBaseline() #ResNet1D_Improved() #ResNet1D()
model.to(device=device)
tqdm.write("Done!\n")

# =============== Define loss function ====================================#
"""
TASK: define the loss; Insert your code here. This can be done in 1 line of code
"""
# Define weighted loss
#loss_function = nn.BCEWithLogitsLoss()
"""
My Design
"""
pos_weight = torch.tensor([7/3], device=device)
loss_function = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

# =============== Define optimizer ========================================#
tqdm.write("Define optimiser...")
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
tqdm.write("Done!\n")

"""
My design
"""
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

# =============== Define lr scheduler =====================================#
# TODO advanced students (non mandatory)
"""
OPTIONAL: define a learning rate scheduler; Insert your code here
"""
from torch.optim.lr_scheduler import StepLR, CosineAnnealingLR, CosineAnnealingWarmRestarts
#lr_scheduler = StepLR(optimizer, step_size=5, gamma=0.1)#None

"""
My design
"""
lr_scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs)
#lr_scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2, eta_min=1e-6)

# Early stopping
#early_stopping = EarlyStopping(patience=10, verbose=True, path="model.pth")

# =============== Train model =============================================#
tqdm.write("Training...")
best_loss = np.Inf

# allocation
train_loss_all, valid_loss_all = [], []

"""
My Design: Add codes for calculating the best threshold based on the F1 scores
"""
all_y_preds = []
all_y_trues = []

# loop over epochs
for epoch in trange(1, num_epochs + 1):
    # training loop
    train_loss = train_loop(epoch, train_dataloader, model, optimizer, loss_function, device)
    # validation loop
    valid_loss, y_pred, y_true = eval_loop(epoch, valid_dataloader, model, loss_function, device)

    """
    My Design
    """
    # Call early stopping for checking
    # early_stopping(valid_loss, model)

    #if early_stopping.early_stop:
    #    print("Early stopping triggered!")
    #    break

    all_y_preds.append(y_pred)
    all_y_trues.append(y_true)

    # collect losses
    train_loss_all.append(train_loss)
    valid_loss_all.append(valid_loss)

    # compute validation metrics for performance evaluation
    """
    TASK: compute validation metrics (e.g. AUROC); Insert your code here
    This can be done e.g. in 5 lines of code
    """
    from sklearn.metrics import roc_auc_score
    valid_auroc = roc_auc_score(y_true, y_pred)
    tqdm.write(f"Validation AUROC: {valid_auroc:.4f}")

    """
    My Designs
    """
    # Calculate Validation Accuracy, F1 score, AP (the default value is 0.5)
    from sklearn.metrics import accuracy_score, f1_score, average_precision_score
    best_threshold = find_best_threshold_by_f1_exact(y_true, y_pred)
    y_pred_label = (y_pred > best_threshold[0]).astype(int)
    valid_acc = accuracy_score(y_true, y_pred_label)
    valid_f1 = f1_score(y_true, y_pred_label)
    valid_ap = average_precision_score(y_true, y_pred)

    tqdm.write(f"Validation Accuracy: {valid_acc:.4f}")
    tqdm.write(f"Validation F1: {valid_f1:.4f}")
    tqdm.write(f"Validation AP: {valid_ap:.4f}")

    if valid_loss < best_loss:
        # Save model parameters
        torch.save({'model': model.state_dict()}, 'model.pth')
        # Update best validation loss
        best_loss = valid_loss
        # statement
        model_save_state = "Best model -> saved"
    else:
        model_save_state = ""

    # Print message
    tqdm.write('Epoch {epoch:2d}: \t'
                'Train Loss {train_loss:.6f} \t'
                'Valid Loss {valid_loss:.6f} \t'
                '{model_save}'
                .format(epoch=epoch,
                        train_loss=train_loss,
                        valid_loss=valid_loss,
                        model_save=model_save_state)
                    )

    # Update learning rate with lr-scheduler
    if lr_scheduler:
        lr_scheduler.step()

"""
TASK: Here it can make sense to plot your learning curve; Insert your code here
"""
import matplotlib.pyplot as plt

plt.plot(train_loss_all, label='Train Loss')
plt.plot(valid_loss_all, label='Valid Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Learning Curve')
plt.show()

"""
My design: Search for the threshold that maximizes the F1 score based on the unique values among all predicted probabilities (y_prob) as the threshold.
"""
best_threshold, best_f1, best_precision, best_recall = find_best_threshold_by_f1_exact(all_y_trues[-1], all_y_preds[-1])
print(f"Best threshold: {best_threshold:.4f}")
print(f"Best F1 score: {best_f1:.4f}, Best precision: {best_precision:.4f}, Best recall: {best_recall:.4f}")

In [ ]:
auroc = roc_auc_score(all_y_trues[-1], all_y_preds[-1])
ap = average_precision_score(all_y_trues[-1], all_y_preds[-1])
print(f"AUROC: {auroc:.4f}, Average Precision (AP):{ap:.4f}")

In [ ]:
# =============== Define model ============================================#
tqdm.write("Define model...")
model = ResNet1D_Improved()
model.to(device=device)
tqdm.write("Done!\n")

# =============== Define loss function ====================================#
pos_weight = torch.tensor([7/3], device=device)
loss_function = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

# =============== Define optimizer ========================================#
tqdm.write("Define optimiser...")
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
tqdm.write("Done!\n")

# =============== Define lr scheduler =====================================#
from torch.optim.lr_scheduler import ReduceLROnPlateau
lr_scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)

# =============== Early stopping ==========================================#
early_stopping = EarlyStopping(patience=8, verbose=True, path="model.pth")

# =============== Train model =============================================#
tqdm.write("Training...")
best_loss = np.Inf
train_loss_all, valid_loss_all = [], []

all_y_preds = []
all_y_trues = []

for epoch in trange(1, num_epochs + 1):
    # training loop
    train_loss = train_loop(epoch, train_dataloader, model, optimizer, loss_function, device)
    # validation loop
    valid_loss, y_pred, y_true = eval_loop(epoch, valid_dataloader, model, loss_function, device)

    # Early stopping
    early_stopping(valid_loss, model)
    if early_stopping.early_stop:
        print("Early stopping triggered!")
        break

    # Append predictions and labels
    all_y_preds.append(y_pred)
    all_y_trues.append(y_true)

    # Collect losses
    train_loss_all.append(train_loss)
    valid_loss_all.append(valid_loss)

    # Compute validation metrics
    from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, average_precision_score
    valid_auroc = roc_auc_score(y_true, y_pred)
    best_threshold = find_best_threshold_by_f1_exact(y_true, y_pred)
    y_pred_label = (y_pred > best_threshold[0]).astype(int)
    valid_acc = accuracy_score(y_true, y_pred_label)
    valid_f1 = f1_score(y_true, y_pred_label)
    valid_ap = average_precision_score(y_true, y_pred)

    tqdm.write(f"Validation AUROC: {valid_auroc:.4f}")
    tqdm.write(f"Validation Accuracy: {valid_acc:.4f}")
    tqdm.write(f"Validation F1: {valid_f1:.4f}")
    tqdm.write(f"Validation AP: {valid_ap:.4f}")

    # Save best model
    if valid_loss < best_loss:
        torch.save({'model': model.state_dict()}, 'model_Reduce.pth')
        best_loss = valid_loss
        model_save_state = "Best model -> saved"
    else:
        model_save_state = ""

    tqdm.write('Epoch {epoch:2d}: \t'
                'Train Loss {train_loss:.6f} \t'
                'Valid Loss {valid_loss:.6f} \t'
                '{model_save}'
                .format(epoch=epoch,
                        train_loss=train_loss,
                        valid_loss=valid_loss,
                        model_save=model_save_state)
                )

    # Update learning rate based on validation loss
    if lr_scheduler:
        lr_scheduler.step(valid_f1)

# =============== Plot learning curve =====================================#
import matplotlib.pyplot as plt
plt.plot(train_loss_all, label='Train Loss')
plt.plot(valid_loss_all, label='Valid Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Learning Curve')
plt.show()

# =============== Best threshold & metrics =================================#
best_threshold, best_f1, best_precision, best_recall = find_best_threshold_by_f1_exact(
    all_y_trues[-1], all_y_preds[-1])
print(f"Best threshold: {best_threshold:.4f}")
print(f"Best F1 score: {best_f1:.4f}, Best precision: {best_precision:.4f}, Best recall: {best_recall:.4f}")


In [ ]:
auroc = roc_auc_score(all_y_trues[-1], all_y_preds[-1])
ap = average_precision_score(all_y_trues[-1], all_y_preds[-1])
print(f"AUROC: {auroc:.4f}, Average Precision (AP):{ap:.4f}")

---
## Model Testing

Since we saved our best model, we can now load the trained model and make predictions on the test data set. We save the predictions in a csv file which will be uploaded as part of the deliverables. Note that we take a `Sigmoid()` function on the model prediction in order to obtain soft predictions (probabilities) instead of hard predictions (0s or 1s).

### Coding Task 7: Make prediction for test data

Here you do not really need to code but you have to:
- replace the baseline model with your model. If you do not use colab then change the path to the model location to load the trained model)
- run the script. The predictions are saved in the variable `soft_pred`.
- upload your predictions to the leaderboard online (see instruction details below).

In [ ]:
# build the dataloader once and re-use when running the cell below possibly multiple times.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# =============== Build data loaders ==========================================#
tqdm.write("Building data loaders...")
# load data
path_to_h5_test, path_to_csv_test = 'codesubset/test.h5', 'codesubset/test.csv'
traces = torch.tensor(h5py.File(path_to_h5_test, 'r')['tracings'][()], dtype=torch.float32)
dataset = TensorDataset(traces)
len_dataset = len(dataset)
# build data loaders
test_dataloader = DataLoader(dataset, batch_size=32, shuffle=False)
tqdm.write("Done!\n")

In [ ]:
# =============== Define model ================================================#
tqdm.write("Define model...")
"""
TASK: Replace the baseline model with your model; Insert your code here
"""
model = ResNet1D_Improved()

# load stored model parameters
ckpt = torch.load('model.pth', map_location=lambda storage, loc: storage)
model.load_state_dict(ckpt['model'])
# put model on device
model.to(device=device)
tqdm.write("Done!\n")

# =============== Evaluate model ==============================================#
model.eval()
# allocation
test_pred = torch.zeros(len_dataset,1)
# progress bar def
test_pbar = tqdm(test_dataloader, desc="Testing")
# evaluation loop
end=0
for traces in test_pbar:
    # data to device
    traces = traces[0].to(device)
    start = end
    with torch.no_grad():
        # Forward pass
        model_output = model(traces)

        # store output
        end = min(start + len(model_output), test_pred.shape[0])
        test_pred[start:end] = torch.nn.Sigmoid()(model_output).detach().cpu()

test_pbar.close()

# =============== Save predictions ============================================#
soft_pred = np.stack((1-test_pred.numpy(), test_pred.numpy()),axis=1).squeeze()

To upload your predictions to the leaderboard, use the following code. There are the following steps to follow:
1. Download the GitHub repository for the leaderboard submission system.
2. Register your team with a **team id** and **password**. The password ensures that only your team can upload to your team id. Do only run the registration once.
3. Upload you predictions as a new submission. There are some things to obey here:
    - For each submission you have to attach a note for you to keep track of the submission in the leaderboard and for us to know which submission you refer to in your explanation. Choose something meaningful such as "submission A" or "model B".
    - You can only get one prediction evaluated per day and you get the score the following day. If you do multiple submissions on the same day, the initial submission will be overwritten and thus only the final submission will be evaluated.
    - Only a maximum of ***FIVE*** submissions will be evaluated. So make them count! (If you update an submission before it is evaluated it doesn't count)
    - The evaluation score is published with you team_id and note at http://hyperion.it.uu.se:5050/leaderboard



In [ ]:
# 1. Download repository for leaderboard submission system
if not exists('leaderboard'):
    !git clone https://gist.github.com/3ff6c4c867331c0bf334301842d753c7.git leaderboard

In [ ]:
# 2. Registration of your team
host = "http://hyperion.it.uu.se:5050/"
runfile("leaderboard/leaderboard_helpers.py")

"""
TASK: Decide for a team_id (max 20 chars) and password.
Do not change this after you have registered your team
"""
team_id = 'YangyangWen_Group25' #Fill in a string
password = 'yyw_group25' #Fill in a string

# run the registration
r = register_team(team_id, password)
if (r.status_code == 201):
    print("Team registered successfully! Good luck")
elif not (r.status_code == 200):
    raise Exception("You can not change your password once created. If you need help, please contact the teachers")
print(r.status_code)
print(r.text)


In [ ]:
# 3. Upload the prediction as submission

# Write a note about the training procedure so you can identify it in the leaderboard. e.g. 5 epochs, or First  (Max 20 characters)
"""
TASK: Add a note for you submission
"""
note = 'Submission 3' #Fill in a string

# Submit the predictions to the leaderboard. Note, this also saves your submissions in your colab folder
r = submit(team_id, password, soft_pred.tolist(), note)
if r.status_code == 201:
    print("Submission successful!")
elif r.status_code == 200:
    print("Submission updated!")

In [ ]:
print(r.status_code)
print(r.text)


### Explanation Task 4: Submissions
One of the grading criteria are three submissions to the leaderboard. List the three main submissions in the table below and explain the main changes in your code for each submission.

<br />


**<font color='red'>Your explanation here:</font>**

Submission 1:
  - After testing multiple CNN architectures (ModelBaseline, Model, LightCNN, ResNet1D, and ResNet1D_Improved), I selected ResNet1D_Improved as it achieved the best results. The architecture includes residual blocks with larger kernels and dropout layers:
    - Layer1: kernel=7, dropout=0.1
    - Layer2: kernel=5, dropout=0.2, stride=2
    - Layer3: kernel=5, dropout=0.3, stride=2
    - Layer4: kernel=3, dropout=0.3, stride=2 (new additional layer)

Submission 2:
- Identified class imbalance in the dataset (7,000 positive vs. 3,000 negative samples). Adjusted the loss function with class weights to address imbalance:
```python
pos_weight = torch.tensor([7/3], device=device)
loss_function = nn.BCEWithLogitsLoss(pos_weight=pos_weight)


Submission 3:
- Experimented with different optimizers (Adam, AdamW) and learning rate schedulers (StepLR, CosineAnnealingLR, CosineAnnealingWarmRestarts). Found that AdamW with CosineAnnealingLR provided slight performance improvements.

Your team id: **<font color='red'>YangyangWen_Group25</font>**

| Submission note | Accuracy | F1 | AUC | AP | Submission description |
| --------------- | -------- | -- | --  | -- | ---------------------- |
|ImprovedResnet1D              | 0.910        | 0.905  | 0.972   | 0.965  | Submission 1                   |
|Imbalanced samples, changed loss function weights              | 0.931        | 0.931  | 0.978   | 0.977  | Submission 2                   |
|Optimizers changes             | 0.938        | 0.938  | 0.980   | 0.978  | Submission 3                   |

### Explanation Task 5: Reflection on Metrics
Your were asked to reach a certain value in AUC and AP while maximising F1 for the leaderboard position. Explain in bullet points what aspect each of the metrics covers and why it is important not to just focus on one metric. What can happen if you only focus on AUC for example?

<br />

**<font color='red'>Your explanation here:</font>**

ROC curve & AUROC
- ROC plots False Positive Rate (x-axis) vs True Positive Rate (y-axis) across different thresholds.
- AUROC is the area under this curve, representing the probability that a randomly chosen positive is ranked higher than a negative.
- Higher AUROC means better separation between positive and negative classes.

Average Precision (AP)
- Computed from the Precision–Recall curve (Precision on y-axis, Recall on x-axis).
- Calculated by sorting predictions by confidence score, evaluating Precision and Recall at multiple thresholds, and taking a weighted average.
- More sensitive to class imbalance than AUROC — low AP indicates poor positive-class performance even when AUROC is high.

F1-score
- Harmonic mean of Precision and Recall, reflecting the trade-off between retrieving positives and avoiding false positives.
- Particularly relevant for imbalanced datasets where accuracy is misleading.

Why not focus on one metric only?
- Optimising only AUROC may yield high ranking ability but very low Precision if the threshold leads to over-predicting positives.
- Balanced evaluation across AUROC, AP, and F1 ensures both ranking quality and usable Precision–Recall trade-offs.

In summary, AUC focuses on overall ranking ability and is not affected by the ratio of positive and negative classes. AP is sensitive to the ratio of positive classes and focuses more on the capture effect of positive classes. F1 is not misled by overall accuracy, but rather exposes the “false high” of accuracy in unbalanced data.